# Data plots Calanus

Plots of the observations, the mask and some histograms based on the observations.      
The interpolation will be performed in notebook [interp_Calanus](interp_Calanus.ipynb).

In [ ]:
using CSV
using Dates
using DelimitedFiles 
using NCDatasets
using PyPlot
using DIVAnd
using TOML
using Statistics
const plt=PyPlot
using PyCall
using PyPlot
mpl = PyCall.pyimport("matplotlib")
ccrs = PyCall.pyimport("cartopy.crs")
cfeature = PyCall.pyimport("cartopy.feature")
coast = cfeature.GSHHSFeature(scale="high");
cartopyticker = PyCall.pyimport("cartopy.mpl.ticker")
lon_formatter = cartopyticker.LongitudeFormatter()
lat_formatter = cartopyticker.LatitudeFormatter()
include("../src/InterpCalanus.jl")
mpl.style.use("calanus.mplstyle")

In [ ]:
makeplot = true
usecartopy = false

## 1. Setting

In [ ]:
config = TOML.parsefile("./config.ini")

### 1.1 Colors, domain, projection 

In [ ]:
makeplot = true
mycolor = "#6667AB"
dom = config["domain"]

if usecartopy
    mainproj = ccrs.Mercator(central_longitude=0.5*(dom[1] + dom[2]),
        min_latitude=dom[3], max_latitude=dom[4])
    datacrs = ccrs.PlateCarree()
end

### 1.2 Files and directories

In [ ]:
datadir = config["directories"]["datadir"]
figdir = config["directories"]["resfigdir"]
resdir = config["directories"]["resdir"]
resdirm = config["directories"]["resdirm"]
isdir(datadir) ? @debug("already there") : mkpath(datadir)
isdir(resdir) ? @debug("already there") : mkpath(resdir)
isdir(resdirm) ? @debug("already there") : mkpath(resdirm)
isdir(figdir) ? @debug("already there") : mkpath(figdir)
datafile = joinpath(datadir, "MBA_CPRdata_Emodnet_21Jan22.csv")
isfile(datafile) ? @debug("already downloaded") : download("https://dox.ulg.ac.be/index.php/s/hjWKf1F3C1Pzz1r/download", datafile)
bathfile = joinpath(datadir, "gebco_30sec_16.nc")
isfile(bathfile) ? @debug("already downloaded") : download("https://dox.ulg.ac.be/index.php/s/U0pqyXhcQrXjEUX/download", bathfile)
isfile(datafile), isfile(bathfile)

### 1.3 Grid configuration

In [ ]:
Δlon = config["dlon"]
Δlat = config["dlat"]
longrid = dom[1]:Δlon:dom[2]
latgrid = dom[3]:Δlat:dom[4]

## 2. Analysis preparation
### 2.1 Metrics

In [ ]:
_, (pm, pn), (xi, yi) = DIVAnd_rectdom(longrid, latgrid);
@info(size(xi));

### Prepare mask

In [ ]:
bx, by, b = DIVAnd.load_bath(bathfile, true, longrid, latgrid)
@show size(b), size(bx), size(by)

In [ ]:
if makeplot
    fig = plt.figure(figsize=(12, 12))
    ax = plt.subplot(111, projection=mainproj)
    ax.set_extend(dom)

    pcm = ax.pcolormesh(bx, by, b', vmin=0., vmax=5000., cmap=PyPlot.cm.gist_earth, transform=datacrs);
    ax.add_feature(coast, facecolor="#363636", edgecolor="k", zorder=5)

    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=true,
                          linewidth=.5, color="gray", alpha=1, linestyle="--")
    gl.top_labels = false
    gl.right_labels = false
    gl.xformatter = lon_formatter
    gl.yformatter = lat_formatter
   
    cb = plt.colorbar(pcm, orientation="vertical", extend="max")
    cb.set_label("Depth (m)", rotation=0, ha="left")
    # plt.savefig(joinpath(figdir, "domain_bathy_050"))
    plt.show()
    # plt.close()
end

In [ ]:
_, _, mask = DIVAnd.load_mask(bathfile, true, longrid, latgrid, 5.0);
# Remove Mediterranean Sea

# - Resolution = 0.1°
# mask[220:end,1:60] .= 0;

# - Resolution = 0.5°
maskindex = findall((xi .>= 0.) .& (yi .<= 47.))
mask[maskindex] .= 0.;

In [ ]:
if makeplot

    fig = plt.figure(figsize=(12, 12))
    ax = plt.subplot(111, projection=mainproj)
    ax.set_extend(dom)

    pcm = ax.pcolormesh(bx, by, mask', cmap=PyPlot.cm.binary, transform=datacrs)
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=true,
        linewidth=.5, color="gray", alpha=1, linestyle="--")
    gl.top_labels = false
    gl.right_labels = false
    gl.xformatter = lon_formatter
    gl.yformatter = lat_formatter

    plt.savefig(joinpath(figdir, "domain_mask_05_5meters"))
    plt.show()
end

## Data reading
### Read the CSV files and separate by columns

In [ ]:
@time lon, lat, dates, calanus_finmarchicus, calanus_helgolandicus = InterpCalanus.read_data_calanus(datafile);

In [ ]:
yearcount, monthcount = InterpCalanus.count_years_months(dates)

### Year histogram

In [ ]:
yearlist = collect(minimum(Dates.year.(dates)):maximum(Dates.year.(dates)));

In [ ]:
if makeplot

    fig = plt.figure(figsize=(12, 8))
    ax = plt.subplot(111)
    ax.set_xlim(yearlist[1] - 0.5, yearlist[end] + 0.5)
    ax.spines["top"].set_visible(false)
    ax.spines["right"].set_visible(false)
    ax.annotate("Min. observations\n($(yearlist[argmin(yearcount)]))", 
        xy=(yearlist[argmin(yearcount)], minimum(yearcount)), 
            xycoords="data",
            xytext=(yearlist[argmin(yearcount)], minimum(yearcount)+ 1100.), textcoords="data",
            horizontalalignment="center", verticalalignment="top",
            )
    plt.bar(yearlist[argmin(yearcount)], minimum(yearcount), color=".2", zorder=2)
    
    ax.text(0, 1, " Max. observations\n($(yearlist[argmax(yearcount)]))", 
        transform=ax.transAxes, 
        horizontalalignment="left", verticalalignment="top")
     

    plt.bar(yearlist[argmax(yearcount)], maximum(yearcount), color=".2", zorder=2)
    
    plt.bar(yearlist, yearcount, color=mycolor)
    plt.savefig(joinpath(figdir, "time_histogram_year"))
    #plt.show()
    plt.close()
end

### Month histogram

In [ ]:
if makeplot
    fig = plt.figure(figsize=(12, 8))
    ax = plt.subplot(111)
    plt.bar(1:12, monthcount, color=mycolor)
    ax.set_title("Distribution of observations by month")
    ax.set_xlim(0.5, 12.5)
    ax.spines["top"].set_visible(false)
    ax.spines["right"].set_visible(false)
    ax.set_xticks(collect(1:12), Dates.monthname.(1:12))
    fig.autofmt_xdate()
    plt.savefig(joinpath(figdir, "time_histogram_month"))
    #plt.show()
    plt.close()
end

### Polar

In [ ]:
if makeplot

    N = 12
    theta = LinRange(0, 2 * π - π/6, N)
    width = (1.8 * π) / N
    fig = plt.figure(figsize=(10, 10))
    ax = plt.subplot(111, polar=true)
    ax.set_thetagrids(collect(0:30:330), Dates.monthname.(1:12), color="k")
    ax.set_theta_zero_location("N")
    bars = ax.bar(theta, monthcount, width=width, color=mycolor)
    plt.savefig(joinpath(figdir, "time_histogram_month_polar"))
    plt.close()
end

## Spatial distribution

In [ ]:
if makeplot

    fig = plt.figure(figsize=(12, 12))
    ax = plt.subplot(111, projection=mainproj)
    ax.set_extend(dom)
    ax.plot(lon, lat, "ko", markersize=1, transform=datacrs)
    ax.add_feature(coast, facecolor="#363636", edgecolor="k", zorder=5)

    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=true,
                          linewidth=.5, color="gray", alpha=1, linestyle="--")
    gl.top_labels = false
    gl.right_labels = false
    gl.xformatter = lon_formatter
    gl.yformatter = lat_formatter
    ax.set_title("Locations of the observations")
    plt.savefig(joinpath(figdir, "location_obs")")
    #plt.show()
    plt.close()
end

### Hexbin maps

In [ ]:
if makeplot

    fig = plt.figure(figsize=(12, 12))
    ax = plt.subplot(111, projection=mainproj)
    ax.set_extend(dom)

    ax.hexbin(lon, lat, transform=datacrs, cmap=plt.cm.hot_r, gridsize=75)
    ax.add_feature(coast, facecolor="#363636", edgecolor="k", zorder=5)

    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=true,
                          linewidth=.5, color="gray", alpha=1, linestyle="--")
    gl.top_labels = false
    gl.right_labels = false
    gl.xformatter = lon_formatter
    gl.yformatter = lat_formatter
    
    ax.set_title("Locations of the observations")
    plt.savefig(joinpath(figdir, "location_hexbin"))
    #plt.show()
    plt.close()
end

### Finmarchicus

In [ ]:
if makeplot
    countmin = 5
    fig = plt.figure(figsize=(12, 12))
    ax = plt.subplot(111, projection=mainproj)
    ax.set_extend(dom)
    ax.scatter(lon, lat, s=.1, lw=.1, color="k", transform=datacrs)
    indexplot = findall(calanus_finmarchicus .>= countmin)
    scat = ax.scatter(lon[indexplot], lat[indexplot], s=1, c=calanus_finmarchicus[indexplot], 
                      cmap=plt.cm.inferno_r, vmin=countmin, vmax=50., transform=datacrs)

    cb = plt.colorbar(scat, extend="both")

    ax.add_feature(coast, facecolor="#363636", edgecolor="k", zorder=5)

    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=true,
                          linewidth=.5, color="gray", alpha=1, linestyle="--")
    gl.top_labels = false
    gl.right_labels = false
    gl.xformatter = lon_formatter
    gl.yformatter = lat_formatter
    ax.set_title("Calanus_finmarchicus observations")

    plt.savefig(joinpath(figdir, "count_calanus_finmarchicus2.jpg"))

    plt.close()
end

In [ ]:
if makeplot
    fig = plt.figure(figsize=(12, 12))
    ax = plt.subplot(111, projection=mainproj)
    ax.set_extent(dom)
    hb = ax.hexbin(lon, lat, C=calanus_finmarchicus, transform=datacrs, cmap=plt.cm.hot_r, gridsize=75)
    cb = plt.colorbar(hb)
    ax.add_feature(coast, facecolor="#363636", edgecolor="k", zorder=5)

    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=true,
                          linewidth=.5, color="gray", alpha=1, linestyle="--")
    gl.top_labels = false
    gl.right_labels = false
    gl.xformatter = lon_formatter
    gl.yformatter = lat_formatter
    gl.xlabel_style = Dict("size" => 10)
    gl.ylabel_style = Dict("size" => 10)
    ax.set_title("Calanus finmarchicus observations")

    plt.savefig(joinpath(figdir, "count_calanus_finmarchicus_hexbin.jpg"))

    plt.close()
end

In [ ]:
if makeplot
    fig = plt.figure(figsize=(12, 12))
    ax = plt.subplot(111, projection=mainproj)
    ax.set_extend(dom)

    hb = ax.hexbin(lon, lat, C=calanus_helgolandicus, transform=datacrs, cmap=plt.cm.hot_r, gridsize=75, 
    vmax=75.)
    cb = plt.colorbar(hb, extend="max")
    ax.add_feature(coast, facecolor="#363636", edgecolor="k", zorder=5)

    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=true,
                          linewidth=.5, color="gray", alpha=1, linestyle="--")
    gl.top_labels = false
    gl.right_labels = false
    gl.xformatter = lon_formatter
    gl.yformatter = lat_formatter
    gl.xlabel_style = Dict("size" => 10)
    gl.ylabel_style = Dict("size" => 10)
    ax.set_title("Calanus helgolandicus observations")

    plt.savefig(joinpath(figdir, "count_calanus_helgolandicus_hexbin.jpg"))

    plt.close()
end

#### Location of zeros

In [ ]:
zerosloc1 = findall(calanus_finmarchicus .== 0.)
lonz1 = lon[zerosloc1]
latz1 = lat[zerosloc1]
zerosloc2 = findall(calanus_helgolandicus .== 0.)
lonz2 = lon[zerosloc2]
latz2 = lat[zerosloc2]

if makeplot

    fig = plt.figure(figsize=(12, 12))
    ax = plt.subplot(111, projection=mainproj)
    ax.set_extend(dom)

    #ax.scatter(0, 0, s=2, lw=.1, alpha=.95, color=mycolor, transform=datacrs, label="0's for Calanus finmarchicus")
    #ax.scatter(0, 0, s=2, lw=.1, alpha=.95, color="#FF9935", transform=datacrs, label="0's for Calanus helgolandicus")
    ax.scatter(lonz1, latz1, s=.5, lw=.1, alpha=.5, color=mycolor, transform=datacrs)
    #ax.scatter(lonz2, latz2, s=.5, lw=.1, alpha=.5, color="#FF9935", transform=datacrs)
    #ax.legend()

    ax.add_feature(coast, facecolor="#363636", edgecolor="k", zorder=5)

    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=true,
                          linewidth=.5, color="gray", alpha=1, linestyle="--")
    gl.top_labels = false
    gl.right_labels = false
    gl.xformatter = lon_formatter
    gl.yformatter = lat_formatter
    gl.xlabel_style = Dict("size" => 10)
    gl.ylabel_style = Dict("size" => 10)
    ax.set_title("Zeros for Calanus finmarchicus")

    plt.savefig(joinpath(figdir, "zeros_calanus_finmarchicus3.jpg"))

    plt.close()

    fig = plt.figure(figsize=(12, 12))
    ax = plt.subplot(111, projection=mainproj)
    ax.scatter(lonz2, latz2, s=.5, lw=.1, alpha=.5, color=mycolor, transform=datacrs)

    ax.add_feature(coast, facecolor="#363636", edgecolor="k", zorder=5)

    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=true,
                          linewidth=.5, color="gray", alpha=1, linestyle="--")
    gl.top_labels = false
    gl.right_labels = false
    gl.xformatter = lon_formatter
    gl.yformatter = lat_formatter
    gl.xlabel_style = Dict("size" => 10)
    gl.ylabel_style = Dict("size" => 10)
    ax.set_title("Zeros for Calanus helgolandicus")

    plt.savefig(joinpath(figdir, "zeros_calanus_helgolandicus3.jpg"))
    plt.close()
end

#### Calanus helgolandicus

In [ ]:
if makeplot

    countmin = 5
    fig = plt.figure(figsize=(12, 12))
    ax = plt.subplot(111, projection=mainproj)
    ax.set_extend(dom)
    ax.scatter(lon[calanus_helgolandicus .> 0], lat[calanus_helgolandicus .> 0], 
        s=.1, lw=.1, color="k", transform=datacrs)
    indexplot = findall(calanus_helgolandicus .>= countmin)
    scat = ax.scatter(lon[indexplot], lat[indexplot], s=1, c=calanus_helgolandicus[indexplot], 
                      cmap=plt.cm.inferno_r, vmin=countmin, vmax=50., transform=datacrs)

    cb = plt.colorbar(scat, extend="both")

    ax.add_feature(coast, facecolor="#363636", edgecolor="k", zorder=5)

    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=true,
                          linewidth=.5, color="gray", alpha=1, linestyle="--")
    gl.top_labels = false
    gl.right_labels = false
    gl.xformatter = lon_formatter
    gl.yformatter = lat_formatter
    gl.xlabel_style = Dict("size" => 10)
    gl.ylabel_style = Dict("size" => 10)
    ax.set_title("Calanus helgolandicus observations")

    plt.savefig(joinpath(figdir, "count_calanus_helgolandicus2.jpg"))

    plt.close()
end